In [1]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
from matplotlib import rcParams
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
# 한글 폰트 설정
rcParams['font.family'] = 'Malgun Gothic'  # 맑은 고딕으로 설정
rcParams['axes.unicode_minus'] = False     # 음수 기호 문제 해결

In [80]:
# EXCEL파일 읽기
file_path = './Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후.xlsx'
df_bigkinds = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [81]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,URL,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사
0,02100101.20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...
1,02100101.20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...
2,02100101.20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...
3,02100101.20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,http://news.mk.co.kr/newsRead.php?year=2017&no...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...
4,02100311.20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,http://www.sedaily.com/NewsView/1OKUY3ELL1,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,02100101.20240821175511003,2024-08-21,매일경제,박재영 기자(jyp8909@mk.co.kr),의족 차고 자전거 국토횡단에 감동 뭉클,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,http://www.mk.co.kr/article/11098054,NaN,"의족 차고 자전거 국토횡단에 감동 뭉클 의족,자전거,국토횡단,감동,뭉클,왼쪽,다리,...",의족 차고 자전거 국토횡단에 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 ...,의족 차 고 자전거 국토횡단 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토 횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리...,족 차고 자전거 국토 횡단 감동 뭉클 족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 족...,의족 자전거 국토 횡단 감동 의족 자전거 국토 횡단 감동 왼쪽 다리 의족 횡단 국토...,족 차고 자전거 국토 횡단 감동 족 자전거 국토 횡단 감동 왼쪽 다리 족 횡단 국토...
4007,02100101.20240822002505001,2024-08-22,매일경제,박재영 기자(jyp8909@mk.co.kr),“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,http://www.mk.co.kr/article/11098294,NaN,"“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’ 횡단,

In [82]:
# KoBERT 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("kwoncho/KoFinBERT")
model = AutoModelForSequenceClassification.from_pretrained("kwoncho/KoFinBERT")

In [83]:
text_classifier = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    truncation=True,  # 긴 텍스트를 자동으로 자름
    max_length=512,   # 최대 길이를 512로 제한
    top_k=None,       # 모든 레이블의 확률 반환
    device=0          # GPU 사용 가능 시 (0), 그렇지 않으면 CPU (-1)
)

In [84]:
max_length = 512

s = time.time()
predicted_scores = []
# tqdm을 사용하여 처리 상태 표시
for text in tqdm(df_bigkinds['제목_키워드_전처리후_Komoran'], desc="Predicting"):
    # 감정 분석 수행
    prediction = text_classifier(text)
    
    # 레이블을 숫자 값으로 매핑
    label_mapping = {
        'positive': 1,
        'neutral': 0,
        'negative': -1
    }
    
    # 연속적인 점수 직접 계산
    score = 0
    for item in prediction[0]:
        label = item['label']
        prob = item['score']
        score += label_mapping[label] * prob
    predicted_scores.append(score)

print("Prediction Time: ", time.time() - s)

Predicting: 100%|██████████████████████████████████████████████████████████████████| 4011/4011 [01:35<00:00, 41.92it/s]

Prediction Time:  95.6931357383728


In [85]:
# 뉴스 식별자 컬럼에서 '.' 이후의 값만 조회
df_bigkinds['뉴스 식별자'] = df_bigkinds['뉴스 식별자'].str.split('.').str[1]

In [86]:
# 데이터프레임에 결과 추가
df_bigkinds['KoFinBERT_Sentiment'] = predicted_scores
# 소수점 1자리로 반올림
df_bigkinds['KoFinBERT_Sentiment'] = df_bigkinds['KoFinBERT_Sentiment'].round(1)
df_bigkinds['KoFinBERT_Sentiment'] = df_bigkinds['KoFinBERT_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,KoFinBERT_Sentiment
0,20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...,0.0
1,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.0
2,20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...,0.0
3,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
4,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4006,20240821175511003,2024-08-21,매일경제,박재영 기자(jyp8909@mk.co.kr),의족 차고 자전거 국토횡단에 감동 뭉클,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,NaN,"의족 차고 자전거 국토횡단에 감동 뭉클 의족,자전거,국토횡단,감동,뭉클,왼쪽,다리,...",의족 차고 자전거 국토횡단에 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 ...,의족 차 고 자전거 국토횡단 감동 뭉클 의족 자전거 국토횡단 감동 뭉클 왼쪽 다리 ...,의족 차고 자전거 국토 횡단 감동 뭉클 의족 자전거 국토 횡단 감동 뭉클 왼쪽 다리...,족 차고 자전거 국토 횡단 감동 뭉클 족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 족...,의족 자전거 국토 횡단 감동 의족 자전거 국토 횡단 감동 왼쪽 다리 의족 횡단 국토...,족 차고 자전거 국토 횡단 감동 족 자전거 국토 횡단 감동 왼쪽 다리 족 횡단 국토...,0.0
4007,20240822002505001,2024-08-22,매일경제,박재영 기자(jyp8909@mk.co.kr),“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’,사회>사회일반,NaN,NaN,사고>스포츠사고,NaN,...,NaN,"“자전거 국토 횡단에 650만명이 울었다”...이 선수 보자마자 ‘뭉클’ 횡단,자전...",자전거 국토 횡단에 650만명이 울었다 ...이 선수 보자마자 뭉클 횡단 자전거 국...,자전거 국토 횡단 650만 명이 울었다 선수 보자마자 뭉클 횡단 자전거 국토 650...,자전거 국토 횡단 650만명 울 었다 서 ㄴ 수 보 뭉클 횡단 자전거 국토 650 ...,자전거 국토 횡단 650 만 명 울 었 다 선수 보 뭉클 횡단 자전거 국토 650 ...,자전거 국토 횡단 650 만 명 울 었 다 선수 보 뭉클 횡단 자전거 국토 650 ...,자전거 국토 횡단 650 선수 횡단 자전거 국토 650 선수 장애 사이클 선수 박찬...,자전거 국토 횡단 명 선수 횡단 자전거 국토 선수 장애 사

전체 분석 후 분석제외 대상과 중복건 제거

In [88]:
df_bigkinds = df_bigkinds[df_bigkinds['분석제외 여부'].isnull()]

In [89]:
df_bigkinds

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,...,분석제외 여부,제목_키워드,제목_키워드_전처리후,제목_키워드_전처리후_Okt,제목_키워드_전처리후_Hannanum,제목_키워드_전처리후_Kkma,제목_키워드_전처리후_Komoran,제목_키워드_전처리후_kiwi,제목_키워드_전처리후_Komoran_명사,KoFinBERT_Sentiment
0,20170901163910001,2017-09-01,매일경제,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일",사회>사건_사고,사회>교육_시험,NaN,범죄>정치>뇌물수수,NaN,...,NaN,"이재용 항소심, 서울고법 형사 13부에 배당 이달중 첫 기일 이재용,항소심,13부,...",이재용 항소심 서울고법 형사 13부에 배당 이달중 첫 기일 이재용 항소심 13부 서...,이재용 항소심 서울 고법 형사 13 부 배당 달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13부 배당 이달 중 첫 기 ㄹ 이재용 항소심 13부 ...,이재 용 항소심 서울 고법 형사 13 부 배당 이달 중 첫 기일 이재 용 항소심 1...,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,이재용 항소심 서울고법 형사 13 배당 이달 기일 이재용 항소심 13 서울고법 형사...,이재용 항소심 서울고법 형사 부 배당 이달 중 기일 이재용 항소심 부 서울고법 형사...,0.0
1,20170901174354001,2017-09-01,매일경제,김제이,최순실게이트 청문회 불출석 11인 중 8명 `백수`,사회>사건_사고,정치>청와대,정치>국회_정당,범죄>정치>뇌물수수,NaN,...,NaN,"최순실게이트 청문회 불출석 11인 중 8명 `백수` 불출석,최순실게이트,청문회,백수...",최순실게이트 청문회 불출석 11인 중 8명 백수 불출석 최순실게이트 청문회 백수 문...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백수 불 출석 최 순 실 게이...,최순실게이트 청문회 불출석 11 ㄴ 중 8명 백수 불출석 최순실게이트 청문회 백수 ...,최 순 실 게이트 청문회 불 출석 11 인 중 8 명 백 수 불 출석 최 순 실 게...,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,최순실 게이트 청문회 출석 11 백수 출석 최순실 게이트 청문회 백수 문고리 안봉근...,최순 실 게이트 청문회 출석 중 명 백수 출석 최순 실 게이트 청문회 백수 문고리 ...,0.0
2,20170901181209001,2017-09-01,매일경제,정주원,`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당,사회>사건_사고,사회>여성,사회>교육_시험,범죄>정치>뇌물수수,NaN,...,NaN,"`삼성 뇌물` 이재용 항소심 `신설` 형사13부 배당 삼성,뇌물,신설,이재용,항소심...",삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사13부 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재 용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재 용 항소심...,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,삼성 뇌물 이재용 항소심 신설 형사 13 배당 삼성 뇌물 신설 이재용 항소심 형사 ...,삼성 뇌물 이재용 항소심 신설 형사 부 배당 삼성 뇌물 신설 이재용 항소심 형사 배...,0.0
3,20170901141935001,2017-09-01,매일경제,정주원,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정,사회>사건_사고,정치>청와대,정치>국회_정당,NaN,NaN,...,NaN,두문불출 10개월만에 `문고리 실세` 이재만 안봉근 법정 나와 청문회불출석 인정 두...,두문불출 10개월만에 문고리 실세 이재만 안봉근 법정 나와 청문회불출석 인정 두문불...,두문불출 10 개월 만에 문 고리 실세 이재만 안봉 근 법정 나와 청문회 불 출석 ...,두문불출 10개월 만에 문고리 실세 이재 만 안봉근 법정 청문회불출석 인정 두문불출...,두문불출 10 개월 만 문고리 실세 이재 만 알 ㄴ 봉 근 법정 나오 청문회 불 출...,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,두문불출 10 개월 문고리 실세 이재만 안봉근 법정 청문회 출석 인정 두문불출 개월...,두문불출 개월 문고리 실세 이재만 안 봉 근 법정 청문회 출석 인정 두문불출 개월 ...,0.0
4,20170901153936001,2017-09-01,서울경제,장주영 기자,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장",정치>국회_정당,사회>사건_사고,정치>청와대,NaN,NaN,...,NaN,"청문회 불출석 혐의, 건강상태-절차적 하자로 '무죄' 주장 혐의,청문회,건강상태,절...",청문회 불출석 혐의 건강상태-절차적 하자로 무죄 주장 혐의 청문회 건강상태 절차적 ...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불출석 혐의 건강상태 - 절차적 하자 무죄 주장 혐의 청문회 건강상태 절차적...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,청문회 출석 혐의 건강 상태 절차 하자 주장 혐의 청문회 건강 상태 절차 하자 무죄...,청문회 출석 혐의 건강 상태 절차 하자 무죄 주장 혐의 청문회 건강 상태 절차 무죄...,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,20240819060124001,2024-08-19,서울경제,공준호 기자,'퇴직금 두둑히 챙겨 퇴사' 벌써 옛말? 은행 안 떠나는 은행원,경제>금융_재테크,사회>노동_복지,경제>유통,NaN,NaN,...,NaN,"'퇴직금 두둑히 챙겨 퇴사' 벌써 옛말? 은행 안 떠나는 은행원 퇴직금,퇴사,옛말,...",퇴직금 두둑히 챙겨 퇴사 벌써 옛말 은행 안 떠나는 은행원 퇴직금 퇴사 옛말 은행 ...,퇴직금 두둑히 챙겨 퇴사 벌써 옛말 은행 안 떠나는 은행원 퇴직금 퇴사 옛말 은행 ...,퇴직금 두둑 히 챙기 퇴사 벌써 옛말 은행 안 떠나 는 은행원 퇴직금 퇴사 옛말 은...,퇴직금 두둑히 챙기 퇴사 벌써 옛말 은행 안 떠나 는 은행원 퇴직금 퇴사 옛말 은행...,퇴직금 두둑히 챙기 퇴사 벌써 옛말 은행 안 떠나 는 은행원 퇴직금 퇴사 옛말 은행...,퇴직금 퇴사 옛말 은행 은행원 퇴직금 퇴사 옛말 은행 은행원 신청 시중 은행 상반기...,퇴직금 퇴사 옛말 은행 은행원 퇴직금 퇴사 옛말 은행 은행원 신청 시중 은행 상반기...,0.0
4005,20240820150930001,2024-08-20,서울경제,정예지 기자,대기업도 고령화 임직원 구성에서 50세 이상이 20대 역전,경제>유통,경제>무역,사회>사회일반,NaN,NaN,...,NaN,"대기업도 고령화 임직원 구성에서 50세 이상이 20대 역전 대기업,고령화,임직원,구...",대기업도 고령화 임직원 구성에서 50세 이상이 20대 역전 대기업 고령화 임직원 구...,대기업 도 고령화 임 직원 구성 50 세 20 대 역전 대기업 고령화 임 직원 구성...,대기업 도 고령화 임직원 구성 50세 20대 역전 대기업 고령화 임직원 구성 50세...,대기업 도 고령화 임직원 구성 50 세 20 대 역전 대기업 고령화 임직원 구성 5...,대기업 도 고령 화 임직원 구성 50 세 20대 역전 대기업 고령 화 임직원 구성 ...,기업 고령 임직원 구성 50 20 역전 기업 고령 임직원 구성 50 역전 임직원 연...,대기업 고령 임직원 구성 세 20대 역전 대기업 고령 임직원 구성 세 역전 임직원 ...,0.0
4006,

뉴스 식별자 최소갑 기준으로 제목_키워드_전처리후 그룹핑

In [91]:
# 그룹핑할 컬럼 리스트
group_columns = ['제목_키워드_전처리후_Komoran']

# 각 그룹에서 뉴스 식별자 컬럼의 최소값을 유지
df_bigkinds2 = df_bigkinds.groupby(group_columns, as_index=False).agg({'뉴스 식별자': 'min'})

In [94]:
df_bigkinds2

,제목_키워드_전처리후_Komoran,뉴스 식별자
0,- 3 주 째 대면 불발 도 이재용 기소 조만간 결정 날 듯 3 주 대면 불발 이재...,20200715150951001
1,- TV PD수첩 암페타민 밀반입 박봄 은 어떻 게 입건 유예 되 었 PD수첩 박봄...,20180425075416001
2,1 078 만 실형 선고 이재용 정면 응시 ㄴ 채 한동안 깊 은 침묵 1 078 실...,20210118151417001
3,1 ㄴ 방송 도 360 도로 실감 게 보 방송 360 도 실감 아프리카TV 삼성전자...,20180329093143001
4,1 노총 잃 은 한국노총 정부 정책 협약 재검토 1 노총 한국노총 정부 재검토 정책...,20200107211129001
...,...,...
3556,휴가 끝 법원 잇달 ㄴ 대형 재판 이달 말 이재용 양승태 선고 휴가 법원 휴가 대형...,20240107175510002
3557,휴가 더보 내달 라는 요구 삼성전자 파업 위기 휴가 요구 삼성전자 파업 위기 임금 ...,20220327154552001
3558,흔들리 는 제조 한국 반도체 공장 사고 10 곳 서 조사 도 넘 은 기업 몰이 권력...,20181018172635002
3559,흔들리 는 제조 한국 생산 10 전 곤두박질치 는데 툭하면 밥그릇 파업 생산 10 ...,20181018172845001


In [103]:
# 필요한 컬럼만 선택하여 두 데이터프레임 병합
big_kinds_sentiment = pd.merge(
    df_bigkinds[['일자','제목_키워드_전처리후_Komoran', '뉴스 식별자','KoFinBERT_Sentiment']],
    df_bigkinds2[['제목_키워드_전처리후_Komoran', '뉴스 식별자']],
    on=['제목_키워드_전처리후_Komoran', '뉴스 식별자'],
    how='inner'
)
big_kinds_sentiment = big_kinds_sentiment[['뉴스 식별자','일자','제목_키워드_전처리후_Komoran','KoFinBERT_Sentiment']]
big_kinds_sentiment

,뉴스 식별자,일자,제목_키워드_전처리후_Komoran,KoFinBERT_Sentiment
0,20170901163910001,2017-09-01,이재용 항소심 서울고법 형사 13 부 배당 이달 중 첫 기일 이재용 항소심 13 부...,0.0
1,20170901174354001,2017-09-01,최순 실 게이트 청문회 불 출석 11 ㄴ 중 8 명 백수 불 출석 최순 실 게이트 ...,0.0
2,20170901181209001,2017-09-01,삼성 뇌물 이재용 항소심 신설 형사 13 부 배당 삼성 뇌물 신설 이재용 항소심 형...,0.0
3,20170901141935001,2017-09-01,두문불출 10 개월 만 문고리 실세 이재만 안 봉 근 법정 나오 청문회 불 출석 인...,0.0
4,20170901153936001,2017-09-01,청문회 불 출석 혐의 건강 상태 - 절차 적 하자 무죄 주장 혐의 청문회 건강 상태...,-0.1
...,...,...,...,...
3556,20240819060124001,2024-08-19,퇴직금 두둑히 챙기 퇴사 벌써 옛말 은행 안 떠나 는 은행원 퇴직금 퇴사 옛말 은행...,0.0
3557,20240820150930001,2024-08-20,대기업 도 고령 화 임직원 구성 50 세 20대 역전 대기업 고령 화 임직원 구성 ...,0.0
3558,20240821175511003,2024-08-21,족 차고 자전거 국토 횡단 감동 뭉클 족 자전거 국토 횡단 감동 뭉클 왼쪽 다리 족...,0.0
3559,20240822002505001,2024-08-22,자전거 국토 횡단 650 만 명 울 었 다 선수 보 뭉클 횡단 자전거 국토 650 ...,0.0


In [105]:
big_kinds_sentiment.to_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_Komoran_KoFinBERT_id.xlsx', index=False)

# 아래코드는 마지막 데이터 미생성으로 인해 생성한 코드로 위의 과정이 오래 걸려 추가
file_path = './Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_Komoran_KoFinBERT_id.xlsx'
big_kinds_sentiment = pd.read_excel(file_path, converters={'뉴스 식별자': str})  # '뉴스 식별자'는 해당 열의 실제 이름으로 대체

In [5]:
big_kinds_sentiment_daily = big_kinds_sentiment.groupby('일자', as_index=False)['KoFinBERT_Sentiment'].mean()
# 소수점 1자리로 반올림
big_kinds_sentiment_daily['KoFinBERT_Sentiment'] = big_kinds_sentiment_daily['KoFinBERT_Sentiment'].round(1)
big_kinds_sentiment_daily['KoFinBERT_Sentiment'] = big_kinds_sentiment_daily['KoFinBERT_Sentiment'].apply(lambda x: 0 if x == -0.0 else x)
# 결과 확인
big_kinds_sentiment_daily

,일자,KoFinBERT_Sentiment
0,2017-09-01,0.0
1,2017-09-04,-1.0
2,2017-09-05,0.0
3,2017-09-06,0.0
4,2017-09-07,-0.2
...,...,...
1384,2024-08-19,-0.2
1385,2024-08-20,0.0
1386,2024-08-21,0.0
1387,2024-08-22,0.0


In [7]:
big_kinds_sentiment_daily = big_kinds_sentiment_daily.rename(columns={'일자': 'date', 'KoFinBERT_Sentiment': 'sentiment'})

In [9]:
big_kinds_sentiment_daily

,date,sentiment
0,2017-09-01,0.0
1,2017-09-04,-1.0
2,2017-09-05,0.0
3,2017-09-06,0.0
4,2017-09-07,-0.2
...,...,...
1384,2024-08-19,-0.2
1385,2024-08-20,0.0
1386,2024-08-21,0.0
1387,2024-08-22,0.0


In [11]:
big_kinds_sentiment_daily.to_excel('./Text_Data/bigkinds/삼성전자_국제_20170901-20240830_전처리후_Komoran_KoFinBERT_id_daily.xlsx', index=False)